In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

c:\Users\snail\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/pawee.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [5]:
name = "Pawee Indulakshana"

In [6]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [7]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.1"

In [8]:
from pydantic import BaseModel
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [20]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."
evaluator_system_prompt += f"""
With this context, evaluate the Agent's latest response and return your evaluation as a valid JSON object with the following structure:
{{
  "is_acceptable": boolean,
  "feedback": string
}}
Ensure the response is valid JSON, contains only the JSON object, and includes no additional text or comments. For example:
{{
  "is_acceptable": true,
  "feedback": "The Agent's response is professional and engaging, aligning well with the provided context."
}}
"""

In [21]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [22]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = ollama.chat.completions.create(model=model_name, messages=messages)
    content = response.choices[0].message.content
    import json
    try:
        parsed = json.loads(content)
        return Evaluation(**parsed)
    except Exception:
        return Evaluation(is_acceptable=False, feedback=content)

In [23]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = ollama.chat.completions.create(model=model_name, messages=messages)
reply = response.choices[0].message.content

In [24]:
reply

"As of my knowledge cutoff, I don't hold any patents. However, I'm proud to have worked on various projects that utilize cutting-edge technologies and innovative solutions. My work focuses on developing real-time object detection models using YOLOv11, achieving 93% accuracy in identifying chickens and eggs from video streams.\n\nIf you'd like more information about my professional endeavors or projects, please feel free to ask."

In [25]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=False, feedback='I would say that the response is generally acceptable, but could be improved. Here\'s my evaluation:\n\n**Strengths:**\n\n* The agent clearly answers the user\'s question.\n* The language is professional and engaging.\n* The agent provides a brief explanation of their work to give context.\n\n**Weaknesses:**\n\n* The agent could provide more direct and concise responses in the future, especially when asked specific questions like "do you hold a patent?"\n* In this case, the agent starts elaborating on their projects which may not be entirely relevant to the initial question. Staying focused on answering the question directly would be helpful.\n\n**Feedback for improvement:**\n\nTo strengthen the response, the agent could aim to provide more direct and concise answers while still being professional and engaging.')

In [26]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [27]:
def chat(message, history):
    if "patent" in message: # just for fun, force pig latin if patent is mentioned
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=model_name, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [28]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's response is not professional as per the given context. The tone used is playful (e.g., 'Oday Orfay!', 'may', 'cay') which does not align with the required professionalism. Additionally, the response seems to deviate from answering a direct question and appears to be overly enthusiastic.
Passed evaluation - returning reply
